## 1. Pytorch Autograd

In [144]:
import torch
torch.__version__

'2.0.1+cpu'

In [145]:
x = torch.randn(3)
y = x + 2

In [146]:
x

tensor([-0.3088,  1.4061, -0.2115])

In [147]:
y

tensor([1.6912, 3.4061, 1.7885])

In [148]:
z = y * y * 3
z

tensor([ 8.5803, 34.8050,  9.5962])

In [149]:
z.mean()

tensor(17.6605)

In [150]:
z.mean().item()

17.660497665405273

In [151]:
a = torch.rand(2, 2, requires_grad=True)
a

tensor([[0.1707, 0.1893],
        [0.0442, 0.1106]], requires_grad=True)

In [152]:
a.requires_grad

True

In [153]:
a.detach()

tensor([[0.1707, 0.1893],
        [0.0442, 0.1106]])

In [154]:
with torch.no_grad():
    print((a**2).requires_grad)
    print(a**2)

False
tensor([[0.0291, 0.0358],
        [0.0020, 0.0122]])


In [155]:
weight = torch.ones(4, requires_grad=True)

for epoch in range(5):
    model_out = (weight*3).sum()
    model_out.backward()

    print(weight.grad)

    with torch.no_grad():
        weight -= 0.1 * weight.grad

    weight.grad.zero_()

print(weight)
print(model_out)

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([-0.5000, -0.5000, -0.5000, -0.5000], requires_grad=True)
tensor(-2.4000, grad_fn=<SumBackward0>)


## 2. Back Propagation

In [156]:
x = torch.tensor(1.0)
y = torch.tensor(2.0)

In [157]:
w = torch.tensor(1.0, requires_grad=True)

In [158]:
y_predict = w * x
loss = (y - y_predict)**2
print(loss)

tensor(1., grad_fn=<PowBackward0>)


In [159]:
loss.backward()
w.grad

tensor(-2.)

In [160]:
with torch.no_grad():
    w -= 0.01 * w.grad
w.grad.zero_() # weight 초기화

tensor(0.)

### Gradient Descent : Manually

In [161]:
import numpy as np

x = np.array([1, 2, 3, 4], dtype=np.float32)
y = np.array([2, 4, 6, 8], dtype=np.float32)

w = 0.0

In [162]:
def forward(x):
    return w * x

In [163]:
def loss(y, y_pred):
    return ((y - y_pred)**2).mean()

In [164]:
def gradient(x, y, y_pred):
    return np.dot(2 * x, y_pred - y).mean()

print(forward(5))

0.0


In [165]:
# training
learning_rate = 0.01
n_iters = 20

for epoch in range(n_iters):
    y_pred = forward(x)
    l = loss(y, y_pred)
    dw = gradient(x, y, y_pred)
    # update weight
    w -= learning_rate * dw

    if epoch % 2 == 0:
        print('epoch : %d w : %.2f loss : %.2f' %(epoch+1, w, l))

epoch : 1 w : 1.20 loss : 30.00
epoch : 3 w : 1.87 loss : 0.77
epoch : 5 w : 1.98 loss : 0.02
epoch : 7 w : 2.00 loss : 0.00
epoch : 9 w : 2.00 loss : 0.00
epoch : 11 w : 2.00 loss : 0.00
epoch : 13 w : 2.00 loss : 0.00
epoch : 15 w : 2.00 loss : 0.00
epoch : 17 w : 2.00 loss : 0.00
epoch : 19 w : 2.00 loss : 0.00


In [166]:
print(w)

1.9999999547004699


### Gradient Descent : Autograd

In [173]:
import torch

x = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

In [174]:
def forward(x):
    return w*x

# def loss(y, y_pred):
#     return ((y_pred - y)**2).mean()

In [175]:
import torch.nn as nn

loss = nn.MSELoss()
optimizer = torch.optim.SGD([w], lr=learning_rate)

In [176]:
# torch training
learning_rate = 0.01
n_iters = 100

for epoch in range(n_iters):
    y_pred = forward(x)
    l = loss(y, y_pred)
    # backward pass
    l.backward()

    # update weight
    # with torch.no_grad():
    #     w -= learning_rate * w.grad
    optimizer.step()
    
    # update 후에 w를 zero화 시킨다
    # w.grad.zero_()
    optimizer.zero_grad()

    if epoch % 5 == 0:
        print(f'epoch : {epoch+1} w : {w.item():.3f} l : {l.item():.3f}')

print(f'Prediction after training : {forward(5).item():.3f}')

epoch : 1 w : 0.300 l : 30.000
epoch : 6 w : 1.246 l : 5.906
epoch : 11 w : 1.665 l : 1.163
epoch : 16 w : 1.851 l : 0.229
epoch : 21 w : 1.934 l : 0.045
epoch : 26 w : 1.971 l : 0.009
epoch : 31 w : 1.987 l : 0.002
epoch : 36 w : 1.994 l : 0.000
epoch : 41 w : 1.997 l : 0.000
epoch : 46 w : 1.999 l : 0.000
epoch : 51 w : 1.999 l : 0.000
epoch : 56 w : 2.000 l : 0.000
epoch : 61 w : 2.000 l : 0.000
epoch : 66 w : 2.000 l : 0.000
epoch : 71 w : 2.000 l : 0.000
epoch : 76 w : 2.000 l : 0.000
epoch : 81 w : 2.000 l : 0.000
epoch : 86 w : 2.000 l : 0.000
epoch : 91 w : 2.000 l : 0.000
epoch : 96 w : 2.000 l : 0.000
Prediction after training : 10.000


### Advanced Training

In [217]:
import torch

x = torch.tensor([1, 2, 3, 4], dtype=torch.float32).reshape(-1, 1)
y = torch.tensor([2, 4, 6, 8], dtype=torch.float32).reshape(-1, 1)

n_samples, n_features = x.shape

In [218]:
x

tensor([[1.],
        [2.],
        [3.],
        [4.]])

In [219]:
x_test = torch.tensor([5], dtype=torch.float32)
x_test

tensor([5.])

In [220]:
input_size = x.shape[1]
output_size = y.shape[1]

In [221]:
# model build
model = nn.Linear(input_size, output_size)

In [222]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define layers
        self.lin = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.lin(x)

In [223]:
model = LinearRegression(input_size, output_size)

In [224]:
learning_rate = 0.01
n_iters = 100

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [225]:
for epoch in range(n_iters):
    y_pred = model(x)
    l = loss(y, y_pred)
    # backward pass
    l.backward()

    # update weight
    # with torch.no_grad():
    #     w -= learning_rate * w.grad
    optimizer.step()
    
    # update 후에 w를 zero화 시킨다
    # w.grad.zero_()
    optimizer.zero_grad()

    if epoch % 5 == 0:
        w, b = model.parameters()
        print(f'epoch : {epoch+1} w : {w[0].item():.3f} l : {l.item():.3f}')

print(f'Prediction after training : {model(x_test).item():.3f}')

epoch : 1 w : 0.275 l : 22.310
epoch : 6 w : 1.049 l : 3.821
epoch : 11 w : 1.363 l : 0.840
epoch : 16 w : 1.493 l : 0.354
epoch : 21 w : 1.549 l : 0.269
epoch : 26 w : 1.575 l : 0.249
epoch : 31 w : 1.589 l : 0.240
epoch : 36 w : 1.598 l : 0.232
epoch : 41 w : 1.605 l : 0.225
epoch : 46 w : 1.612 l : 0.219
epoch : 51 w : 1.618 l : 0.212
epoch : 56 w : 1.623 l : 0.206
epoch : 61 w : 1.629 l : 0.200
epoch : 66 w : 1.634 l : 0.194
epoch : 71 w : 1.640 l : 0.188
epoch : 76 w : 1.645 l : 0.183
epoch : 81 w : 1.651 l : 0.177
epoch : 86 w : 1.656 l : 0.172
epoch : 91 w : 1.661 l : 0.167
epoch : 96 w : 1.666 l : 0.162
Prediction after training : 9.320
